# 7506 - Trabajo Práctico 2

---

## Introducción

### Librerías

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [2]:
# Dataset Train preprocesado
train_ds = pd.read_csv('datasets/tp1-train.csv')
train_ds = train_ds.drop(['Unnamed: 0'], axis=1)
train_ds.head()

,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,2021-03-31,2021-05-01 00:00:00.000000000,-34.635768,-58.494408,Floresta,PH,2.0,1.0,58.0,56.0,107000.0,VENTA PH FLORESTA DOS AMBIENTES APTO PROFESIONAL
1,2021-08-25,2021-11-07 07:43:19.924328415,-34.569809,-58.447177,Colegiales,Departamento,4.0,3.0,91.0,80.0,270000.0,Departamento venta 4 ambientes cochera baulera...
2,2021-08-26,2021-10-14 00:00:00.000000000,-34.623384,-58.436355,Caballito,Departamento,4.0,3.0,80.0,75.0,170000.0,Departamento - Caballito
3,2021-05-28,2021-06-03 00:00:00.000000000,-34.590503,-58.408862,Palermo,Departamento,4.0,3.0,67.0,67.0,212000.0,Departamento - Palermo
4,2021-09-08,2021-09-26 00:00:00.000000000,-34.628205,-58.374628,Barracas,Departamento,3.0,2.0,52.0,52.0,115200.0,VENTA DEPARTAMENTO DE 3 AMBIENTES en SAN TELMO


In [3]:
# Dataset Test preprocesado
test_ds = pd.read_csv('datasets/tp1-test.csv')
test_ds = test_ds.drop(['Unnamed: 0'], axis=1)
test_ds

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,-34.626705,-58.385174,San Cristobal,Departamento,4.0,3.0,75.0,71.0,75000.0,VENTA DPTO 3 AMB C/DEPENDENCIA - CONSTITUCION
1,-34.557145,-58.466550,Nuñez,Departamento,1.0,1.0,40.0,40.0,76000.0,Venta en pozo monoambiente amplio en Nuñez
2,-34.603738,-58.398443,Balvanera,Departamento,3.0,2.0,81.0,70.0,115000.0,Departamento - Balvanera - 3 ambientes - Depen...
3,-34.614708,-58.418004,Almagro,Departamento,3.0,2.0,51.0,42.0,110000.0,VENTA TRES AMBIENTES ALMAGRO
4,-34.567463,-58.449964,Belgrano,Departamento,2.0,1.0,55.0,40.0,99000.0,Departamento venta
...,...,...,...,...,...,...,...,...,...,...
10624,-34.561818,-58.467789,Belgrano,Departamento,3.0,2.0,76.0,70.0,217000.0,Exc S/Piso 3 amb c/2 bcones Gr Vta Panor Coc c...
10625,-34.610377,-58.447886,Caballito,Departamento,2.0,1.0,48.0,41.0,110000.0,Departamento - Caballito Norte
10626,-34.615916,-58.418322,Almagro,PH,4.0,3.0,100.0,100.0,177000.0,PH - Almagro
10627,-34.587055,-58.405742,Recoleta,Departamento,2.0,1.0,35.0,35.0,85000.0,French 3000


## Procesamiento del lenguaje natural

### Ampliación del dataset

Este dataset incluye descripciones de las propiedades del otro dataset. Veremos como podemos extraer información de estas descripciones.

In [4]:
descriptions_dataset = pd.read_csv('datasets/properati_argentina_2021_decrip.csv')

In [5]:
descriptions_dataset

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...
...,...,...
460149,MbQ4rqEzjp9Jfc3Sywb/YQ==,Corredor Responsable: Hector Garaygorta / Cris...
460150,fbuG1ohYC623nxLYPQPBvw==,Corredor Responsable: MARIA DEL ROSARIO NEME -...
460151,Ry5hMwctFXD7rLoUpzduxQ==,En cumplimiento de las leyes provinciales vige...
460152,xH37f+l+VOP8Vj0T3g+r6g==,En cumplimiento de las leyes provinciales vige...


#### Tecnica Minqing Hu y Bing Liu

#### Tecnica Regex

Revisamos las siguientes paginas para entender cuales son los ammenities mas buscados en CABA, y en Argentina en general.

https://www.iprofesional.com/negocios/371702-cuales-son-los-amenities-mas-exoticos-de-edificios-en-argentina

https://www.baenegocios.com/sociedad/Ranking-de-amenities-los-servicios-que-mas-pesan-al-comprar-una-propiedad-20220119-0068.html

https://www.forbesargentina.com/negocios/amenities-servicios-mas-demandados-argentinos-comprar-una-propiedad-n11901

Dichos ammenities parecen hacer que la propiedad cotice entre un 15% y un 20% más que el precio de venta.

Sacando un promedio y haciendo un top-5 ranking, podemos notar que los mas relevantes son:

- Garage/Estacionamiento
- Pileta
- Jardin/Espacio al aire libre
- Parrilla
- SUM (Gimnasio/Spa/Sauna)

Al buscar estos datos, podriamos tratar de entender si el precio resulta mayor, contra una propiedad de similares caracteristicas pero sin estos ammenities y a partir de eso, entender que % varía del precio de venta original.

In [6]:
garage_sin_espacio = re.compile(r"[^\s]+garage|garaje|estacionamiento|parking")
garage_con_espacio = re.compile(r"[^\s]+ garage|garaje|estacionamiento|parking")

In [7]:
frecuencia_garage = {'garage': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(garage_sin_espacio, str(desc).lower())) > 0):
        frecuencia_garage['garage'] += 1
    if (len(re.findall(garage_con_espacio, str(desc).lower())) > 0):
        frecuencia_garage['garage'] += 1
print(frecuencia_garage)

{'garage': 65719}


In [8]:
pileta_sin_espacio = re.compile(r"[^\s]+pileta")
pileta_con_espacio = re.compile(r"[^\s]+ pileta")

In [9]:
frecuencia_pileta= {'pileta': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(pileta_sin_espacio, str(desc).lower())) > 0):
        frecuencia_pileta['pileta'] += 1
    if (len(re.findall(pileta_con_espacio, str(desc).lower())) > 0):
        frecuencia_pileta['pileta'] += 1
print(frecuencia_pileta)

{'pileta': 81058}


In [10]:
jardin_sin_espacio = re.compile(r"[^\s]+jardin|espacio verde")
jardin_con_espacio = re.compile(r"[^\s]+ jardin|espacio verde")

In [11]:
frecuencia_jardin = {'jardin': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(jardin_sin_espacio, str(desc).lower())) > 0):
        frecuencia_jardin['jardin'] += 1
    if (len(re.findall(jardin_con_espacio, str(desc).lower())) > 0):
        frecuencia_jardin['jardin'] += 1
print(frecuencia_jardin)

{'jardin': 33055}


In [12]:
parrilla_sin_espacio = re.compile(r"[^\s]+parrilla|bbq")
parrilla_con_espacio = re.compile(r"[^\s]+ parrilla|bbq")

In [13]:
frecuencia_parrilla = {'parrilla': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(parrilla_sin_espacio, str(desc).lower())) > 0):
        frecuencia_parrilla['parrilla'] += 1
    if (len(re.findall(parrilla_con_espacio, str(desc).lower())) > 0):
        frecuencia_parrilla['parrilla'] += 1
print(frecuencia_parrilla)

{'parrilla': 119147}


In [14]:
zoom_sin_espacio = re.compile(r"[^\s]+zoom|sum|gimansio|spa")
zoom_con_espacio = re.compile(r"[^\s]+ zoom|sum|gimansio|spa")

In [15]:
frecuencia_zoom = {'sum': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(zoom_sin_espacio, str(desc).lower())) > 0):
        frecuencia_zoom['sum'] += 1
    if (len(re.findall(zoom_con_espacio, str(desc).lower())) > 0):
        frecuencia_zoom['sum'] += 1
print(frecuencia_zoom)

{'sum': 337200}


Notamos que muchas obervaciones tenian el dato de si contaban o no con balcon asi que decidimos agregarlo al analisis para poder ver si encontramos una correlacion que nos de mas informacion con lo que respecta al balcon y a la superifice total versus la superificie cubierta y el precio de venta de cada vivienda.

In [16]:
balcon_sin_espacio = re.compile(r"[^\s]+balcon")
balcon_con_espacio = re.compile(r"[^\s]+ balcon")

In [17]:
frecuencia_balcon= {'balcon': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(balcon_sin_espacio, str(desc).lower())) > 0):
        frecuencia_balcon['balcon'] += 1
    if (len(re.findall(balcon_con_espacio, str(desc).lower())) > 0):
        frecuencia_balcon['balcon'] += 1
print(frecuencia_balcon)

{'balcon': 46510}


Otro factor interesante que podriamos analizar es a que tipo de vivienda pertenece cada ammenity. Y si donde encontramos una ammenity en particular, encontramos consecuentemente otra. Por ejemplo, una casa con jardin y parrila y/o pileta.
De esta manera podriamos tratar de determinar el costo de cada ammenity o como afecta al precio.

#### Tecnica Open Information Extraction

#### Modelos

## Redes Neuronales

### Regresión

### Clasificación

#### Construcción del target

#### Modelo

## Ensambles de modelos

### Ensamble 1

### Ensamble 2

## Conclusiones